In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_1_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_1_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_1.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_1_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_1_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_1.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_1_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_1_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_1_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_1_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_1_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_1_features.csv')

For BBBP (Classification) \
Valid 2 \
No missing in chemberta2 and molformer \
3 missing in molbert \
Test \
no missing in chemberta2, 1 missing in molformer, 2 missing in molbert





In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8731707317073171,
  'F1 Score': 0.9133333333333333,
  'ROC-AUC': 0.9536774193548387,
  'PR-AUC': 0.9855666168892795},
 'Molformer': {'Accuracy': 0.8823529411764706,
  'F1 Score': 0.9235668789808917,
  'ROC-AUC': 0.8681168831168831,
  'PR-AUC': 0.9131281577368526},
 'Molbert': {'Accuracy': 0.8916256157635468,
  'F1 Score': 0.930379746835443,
  'ROC-AUC': 0.9509011396766499,
  'PR-AUC': 0.9829668845173022}}

In [3]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
# Find the indices of these missing 'smiles' in the chemberta2_valid2 DataFrame
missing_indices_molformer_valid2 = bbbp_chemberta2_valid2.index[bbbp_chemberta2_valid2['smiles'].isin(missing_smiles_molformer_valid2)].tolist()

print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")
print(f"Number of missing indices in molformer_valid2: {len(missing_indices_molformer_valid2)}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
# Find the indices of these missing 'smiles' in the chemberta2_valid2 DataFrame
missing_indices_molbert_valid2 = bbbp_chemberta2_valid2.index[bbbp_chemberta2_valid2['smiles'].isin(missing_smiles_molbert_valid2)].tolist()

print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")
print(f"Number of missing indices in molbert_valid2: {len(missing_indices_molbert_valid2)}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_indices_valid2 = list(set(missing_indices_molformer_valid2).union(set(missing_indices_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_indices_valid2}")

Missing smiles in molformer_valid2: set()
Number of missing indices in molformer_valid2: 0
Missing smiles in molbert_valid2: {'[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3', '[Na+].[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](C([O-])=O)c3cscc3)C(=O)N2[C@H]1C([O-])=O', '[Na+].[Na+].[Na+].[O-]C(=O)[P]([O-])([O-])=O'}
Number of missing indices in molbert_valid2: 3
These indices will be removed from the valid2 set: [267, 189, 247]


In [4]:
bbbp_chemberta2_valid2 = bbbp_chemberta2_valid2.drop(combined_invalid_indices_valid2)
bbbp_molformer_valid2 = bbbp_molformer_valid2.drop(combined_invalid_indices_valid2)
bbbp_chemberta2_features_valid2 = bbbp_chemberta2_features_valid2.drop(combined_invalid_indices_valid2)
bbbp_molformer_features_valid2 = bbbp_molformer_features_valid2.drop(combined_invalid_indices_valid2)
bbbp_molbert_features_valid2 = bbbp_molbert_features_valid2.drop(combined_invalid_indices_valid2)

# check shapes
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(407, 8)
(407, 6)
(407, 4)
(407, 386)
(407, 769)
(407, 769)


In [5]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
missing_indices_molformer_test = bbbp_chemberta2_test.index[bbbp_chemberta2_test['smiles'].isin(missing_smiles_molformer_test)].tolist()

print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")
print(f"Number of missing indices in molformer_test: {len(missing_indices_molformer_test)}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
missing_indices_molbert_test = bbbp_chemberta2_test.index[bbbp_chemberta2_test['smiles'].isin(missing_smiles_molbert_test)].tolist()

print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")
print(f"Number of missing indices in molbert_test: {len(missing_indices_molbert_test)}")

# Combine the invalid indices from molformer_test with the missing indices from molbert_test
combined_invalid_indices_test = list(set(missing_indices_molformer_test).union(set(missing_indices_molbert_test)))

print(f"These indices will be removed from the test set: {combined_invalid_indices_test}")

Missing smiles in molformer_test: {'s1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N'}
Number of missing indices in molformer_test: 1
Missing smiles in molbert_test: {'[Na+].C[C@@H](O)[C@@H]1[C@H]2SC(=C(N2C1=O)C([O-])=O)COC(N)=O', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N'}
Number of missing indices in molbert_test: 2
These indices will be removed from the test set: [42, 94]


In [6]:
bbbp_chemberta2_test = bbbp_chemberta2_test.drop(combined_invalid_indices_test)
bbbp_molformer_test = bbbp_molformer_test.drop(combined_invalid_indices_test)
bbbp_chemberta2_features_test = bbbp_chemberta2_features_test.drop(combined_invalid_indices_test)
bbbp_molformer_features_test = bbbp_molformer_features_test.drop(combined_invalid_indices_test)
bbbp_molbert_features_test = bbbp_molbert_features_test.drop(combined_invalid_indices_test)

# check shapes for test sets
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(203, 8)
(202, 6)
(203, 4)
(203, 386)
(203, 769)
(203, 769)


In [7]:
# create a new dataframe with one column of bbbp_chemberta2_valid2['softmax_class_1_prob']
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of bbbp_molformer_valid2['Prob_Class_1']
bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of bbbp_molbert_valid2['Probabilities']
bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# concatenate the three dataframes
bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)
bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)
bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# combine the features and probabilities
bbbp_X_ensemble_valid2 = pd.concat([bbbp_prob, bbbp_features], axis=1)

bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

In [8]:
# do the same for test probs and features
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)
bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)
bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)
bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)
bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)
bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)
bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

bbbp_X_ensemble_test = pd.concat([bbbp_prob, bbbp_features], axis=1)

# remove the rows with missing values and record indices so it can be removed from the target
missing_indices = bbbp_X_ensemble_test.index[bbbp_X_ensemble_test.isnull().any(axis=1)].tolist()
bbbp_X_ensemble_test = bbbp_X_ensemble_test.drop(missing_indices)
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np'].drop(missing_indices)

In [9]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)

# transform back to dataframe
bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)
bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)


In [10]:
# # use min-max scaling
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
# bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)

# bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)
# bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)

In [11]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# # Define the model with ridge penalty (l2)
# ridge_model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=0)

# # Prepare a range of alpha values to test (or C values, which are the inverse of alpha)
# alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]  # Fewer points, covering a broad range

# # Convert alphas to Cs for the parameter grid (since C is the inverse of alpha)
# Cs = [1/alpha for alpha in alphas]
# params = {'C': Cs}
# grid_search = GridSearchCV(ridge_model, param_grid=params, cv=5, scoring='roc_auc')

# # Fit the grid search to the data
# grid_search.fit(bbbp_X_ensemble_valid2, bbbp_y_ensemble_valid2)

# # Best model after grid search
# bbbp_best_ridge_model = grid_search.best_estimator_

# # Predict the test set
# bbbp_ridge_pred = bbbp_best_ridge_model.predict(bbbp_X_ensemble_test)
# bbbp_ridge_probs = bbbp_best_ridge_model.predict_proba(bbbp_X_ensemble_test)[:, 1]

# # Calculate the metrics
# bbbp_ridge_metrics = {
#     "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_ridge_pred),
#     "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_ridge_pred),
#     "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_ridge_probs),
#     "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_ridge_probs)
# }

# bbbp_ridge_metrics

In [12]:
# use lasso regression to train the ensemble model
from sklearn.linear_model import LogisticRegressionCV

# cross validation for strength of regularization
lasso_cv = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', max_iter=5000, random_state=0, scoring='roc_auc')

# Fit the model
lasso_cv.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_lasso_pred = lasso_cv.predict(bbbp_X_ensemble_test_scaled)
bbbp_lasso_probs = lasso_cv.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
bbbp_lasso_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_lasso_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_lasso_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_lasso_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_lasso_probs)
}

bbbp_lasso_metrics

{'Accuracy': 0.9207920792079208,
 'F1 Score': 0.9483870967741935,
 'ROC-AUC': 0.9639855942376951,
 'PR-AUC': 0.9889951791093342}

In [13]:
coefs = pd.Series(lasso_cv.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

# Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
for model in ['chemberta2', 'molformer', 'molbert']:
    if model not in selected_features:
        selected_features.append(model)

print("Selected Features:", selected_features)
# check how many features are selected
print(len(selected_features))

Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_236', 'chemberta2_feature_379', 'molformer_feature_22', 'molformer_feature_141', 'molformer_feature_312', 'molformer_feature_340', 'molformer_feature_359', 'molformer_feature_537', 'molformer_feature_634']
12


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Define the model with elasticnet penalty
elastic_net_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=5000, random_state=0)

# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

# Convert alphas to Cs for the parameter grid (since C is the inverse of alpha)
Cs = [1/alpha for alpha in alphas]

# Create a more concise grid search using 5-fold cross-validation
params = {
    'C': Cs,
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(bbbp_X_ensemble_valid2_scaled, bbbp_y_ensemble_valid2)

# Best model after grid search
bbbp_best_elastic_model = grid_search.best_estimator_
print(grid_search.best_params_)

# Predict the test set
bbbp_elastic_pred = bbbp_best_elastic_model.predict(bbbp_X_ensemble_test_scaled)
bbbp_elastic_probs = bbbp_best_elastic_model.predict_proba(bbbp_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
bbbp_elastic_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_elastic_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_elastic_probs),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_elastic_probs)
}

bbbp_elastic_metrics

{'C': 0.3333333333333333, 'l1_ratio': 0.9}


{'Accuracy': 0.9207920792079208,
 'F1 Score': 0.9483870967741935,
 'ROC-AUC': 0.9641189809257037,
 'PR-AUC': 0.9896831954808077}

In [15]:
# Access the coefficients from elastic net
coefs = pd.Series(bbbp_best_elastic_model.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

# Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
for model in ['chemberta2', 'molformer', 'molbert']:
    if model not in selected_features:
        selected_features.append(model)

print("Selected Features:", selected_features)
print(len(selected_features))

Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_6', 'chemberta2_feature_24', 'chemberta2_feature_33', 'chemberta2_feature_73', 'chemberta2_feature_74', 'chemberta2_feature_88', 'chemberta2_feature_91', 'chemberta2_feature_127', 'chemberta2_feature_133', 'chemberta2_feature_140', 'chemberta2_feature_146', 'chemberta2_feature_266', 'chemberta2_feature_286', 'chemberta2_feature_307', 'chemberta2_feature_313', 'chemberta2_feature_353', 'chemberta2_feature_379', 'molformer_feature_15', 'molformer_feature_22', 'molformer_feature_51', 'molformer_feature_53', 'molformer_feature_68', 'molformer_feature_96', 'molformer_feature_97', 'molformer_feature_101', 'molformer_feature_102', 'molformer_feature_136', 'molformer_feature_172', 'molformer_feature_176', 'molformer_feature_200', 'molformer_feature_202', 'molformer_feature_205', 'molformer_feature_228', 'molformer_feature_239', 'molformer_feature_245', 'molformer_feature_246', 'molformer_feature_248', 'molformer_feat

In [16]:
# # Access the coefficients from lasso
# coefs = pd.Series(lasso_cv.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# # Filter to get the selected features
# selected_features = coefs[coefs != 0].index.tolist()

# # Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
# for model in ['chemberta2', 'molformer', 'molbert']:
#     if model not in selected_features:
#         selected_features.append(model)

# # Filter the original DataFrame to keep only selected features
# bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2_scaled[selected_features]
# bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test_scaled[selected_features]

# print("Selected Features:", selected_features)
# print("Filtered Dataset Shape:", bbbp_X_ensemble_valid2_selected.shape)


Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_236', 'chemberta2_feature_379', 'molformer_feature_22', 'molformer_feature_141', 'molformer_feature_312', 'molformer_feature_340', 'molformer_feature_359', 'molformer_feature_537', 'molformer_feature_634']
Filtered Dataset Shape: (407, 12)


In [26]:
# Access the coefficients from elastic net
coefs = pd.Series(bbbp_best_elastic_model.coef_[0], index=bbbp_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

# Check if 'chemberta2', 'molformer', 'molbert' are in the selected features, if not, add them
for model in ['chemberta2', 'molformer', 'molbert']:
    if model not in selected_features:
        selected_features.append(model)

# Filter the original DataFrame to keep only selected features
bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2_scaled[selected_features]
bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test_scaled[selected_features]

print("Selected Features:", selected_features)
print("Filtered Dataset Shape:", bbbp_X_ensemble_valid2_selected.shape)

# Now bbbp_X_ensemble_valid2_selected contains only the features selected by LassoCV

Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_6', 'chemberta2_feature_24', 'chemberta2_feature_33', 'chemberta2_feature_73', 'chemberta2_feature_74', 'chemberta2_feature_88', 'chemberta2_feature_91', 'chemberta2_feature_127', 'chemberta2_feature_133', 'chemberta2_feature_140', 'chemberta2_feature_146', 'chemberta2_feature_266', 'chemberta2_feature_286', 'chemberta2_feature_307', 'chemberta2_feature_313', 'chemberta2_feature_353', 'chemberta2_feature_379', 'molformer_feature_15', 'molformer_feature_22', 'molformer_feature_51', 'molformer_feature_53', 'molformer_feature_68', 'molformer_feature_96', 'molformer_feature_97', 'molformer_feature_101', 'molformer_feature_102', 'molformer_feature_136', 'molformer_feature_172', 'molformer_feature_176', 'molformer_feature_200', 'molformer_feature_202', 'molformer_feature_205', 'molformer_feature_228', 'molformer_feature_239', 'molformer_feature_245', 'molformer_feature_246', 'molformer_feature_248', 'molformer_feat

In [27]:
# bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2
# bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test
# check shapes
print(bbbp_X_ensemble_valid2_selected.shape)
print(bbbp_X_ensemble_test_selected.shape)

(407, 125)
(202, 125)


In [28]:
# Initialize and train the SVM model
from sklearn.svm import SVC

bbbp_svm_model = SVC(probability=True)
bbbp_svm_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_svm_pred = bbbp_svm_model.predict(bbbp_X_ensemble_test_selected)
bbbp_svm_probs = bbbp_svm_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_svm_metrics = {
    'Accuracy': accuracy_score(bbbp_y_ensemble_test, bbbp_svm_pred),
    'F1 Score': f1_score(bbbp_y_ensemble_test, bbbp_svm_pred),
    'ROC-AUC': roc_auc_score(bbbp_y_ensemble_test, bbbp_svm_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_svm_probs[:, 1])
}

bbbp_svm_metrics

{'Accuracy': 0.900990099009901,
 'F1 Score': 0.9358974358974359,
 'ROC-AUC': 0.9662531679338402,
 'PR-AUC': 0.9894454706655296}

In [29]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

bbbp_rf_model = RandomForestClassifier(random_state=0)

bbbp_rf_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_rf_best_pred = bbbp_rf_model.predict(bbbp_X_ensemble_test_selected)
bbbp_rf_best_probs = bbbp_rf_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_rf_best_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_rf_best_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_rf_best_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_rf_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_rf_best_probs[:, 1])
}

bbbp_rf_best_metrics

{'Accuracy': 0.900990099009901,
 'F1 Score': 0.9363057324840764,
 'ROC-AUC': 0.9673202614379085,
 'PR-AUC': 0.9896146875969918}

In [30]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Define the hyperparameter space using continuous distributions
bbbp_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = xgb.XGBClassifier(**params, random_state=0)
    
    # Cross-validated AUC score as the objective
    roc_auc = make_scorer(roc_auc_score, response_method='predict_proba')
    score = cross_val_score(model, bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2, scoring=roc_auc, cv=5)
    
    # Minimize the negative ROC AUC score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
bbbp_xgb_best_params = fmin(fn=objective, 
                          space=bbbp_xgb_hyperopt_space, 
                          algo=tpe.suggest, 
                          max_evals=50, 
                          trials=trials,
                          early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_xgb_best_params)

 24%|██▍       | 12/50 [00:07<00:23,  1.62trial/s, best loss: -0.9748587491998109]
Best hyperparameters: {'colsample_bytree': 0.7538067024261862, 'learning_rate': 0.2769809980351193, 'max_depth': 6.0, 'n_estimators': 100.0, 'subsample': 0.6018828374539114}


In [31]:
# Convert parameters obtained from Hyperopt to the correct data type
bbbp_xgb_best_params['n_estimators'] = int(bbbp_xgb_best_params['n_estimators'])
bbbp_xgb_best_params['max_depth'] = int(bbbp_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
bbbp_xgb_model = xgb.XGBClassifier(**bbbp_xgb_best_params, random_state=0)
bbbp_xgb_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_xgb_best_pred = bbbp_xgb_model.predict(bbbp_X_ensemble_test_selected)
bbbp_xgb_best_probs = bbbp_xgb_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_xgb_best_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1])
}

bbbp_xgb_best_metrics

{'Accuracy': 0.905940594059406,
 'F1 Score': 0.9385113268608414,
 'ROC-AUC': 0.9709217020141391,
 'PR-AUC': 0.9913903246048449}

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = bbbp_X_ensemble_valid2_selected
y = bbbp_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
bbbp_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_nn_best_params)


 28%|██▊       | 14/50 [03:06<08:00, 13.35s/trial, best loss: -0.9958044736879937]
Best hyperparameters: {'dropout_rate': 0.041171818543946415, 'learning_rate': 0.00047499175087724153, 'num_layers': 1.0, 'num_neurons': 129.0}


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
bbbp_nn_best_params = {
    'num_layers': int(bbbp_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(bbbp_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': bbbp_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': bbbp_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(bbbp_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(bbbp_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(bbbp_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(bbbp_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=bbbp_X_ensemble_valid2_selected.shape[1], num_layers=bbbp_nn_best_params['num_layers'], 
                 num_neurons=bbbp_nn_best_params['num_neurons'], dropout_rate=bbbp_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=bbbp_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    bbbp_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

bbbp_nn_metrics

{'Accuracy': 0.8762376237623762,
 'F1 Score': 0.9174917491749175,
 'ROC-AUC': 0.9515806322529011,
 'PR-AUC': 0.9848393859440531}

In [34]:
# report all the metrics for ct
bbbp_metrics_results["Elastic Net"] = bbbp_elastic_metrics
bbbp_metrics_results["LASSO"] = bbbp_lasso_metrics
# bbbp_metrics_results["Ridge"] = bbbp_ridge_metrics
bbbp_metrics_results["SVM"] = bbbp_svm_metrics
bbbp_metrics_results["Random Forest"] = bbbp_rf_best_metrics
bbbp_metrics_results["XGBoost"] = bbbp_xgb_best_metrics
bbbp_metrics_results["Neural Network"] = bbbp_nn_metrics

bbbp_metrics_df = pd.DataFrame(bbbp_metrics_results).T

# keep 3 digits after the decimal point
bbbp_metrics_df = bbbp_metrics_df.round(3)

# export as csv
bbbp_metrics_df.to_csv('./split1_bbbp_metrics_elasticFeatures.csv')